Решение системы линейных уравнений методом Гаусса. 

Материалы к вопросу https://ru.stackoverflow.com/questions/1271321

In [1]:
import numpy as np


In [2]:
matrix = np.array([[3.8, 6.7, -1.2, 5.2], 
                   [6.4, 1.3, -2.7, 3.8], 
                   [2.4, -4.5, 3.5, -0.6]])

withZero = np.array([[1,0,0, 1],
                     [0,0,1, 2],
                     [0,1,0, 3]], dtype=float)

# Наивная реализация
Наивная реализация метода Гаусса приведения к треугольной форме. Сломается на матрицах, содержащих нуль на диагонали. Например, на матрице с такими коэффициентами:
```
array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])
```

Функция принимает на вход матрицу `(N+1)xN` - в последней колонке свободные члены. Функция меняет матрицу, переданную в аргументе, поэтому если хочется сохранить матрицу, то вызывать нужно с `np.copy`: `gaussFunc(matrix.copy())`

In [3]:
def makeTriangleNaive(matrix):
    # функция меняет матрицу через побочные эффекты
    # если вам нужно сохранить прежнюю матрицу, скопируйте её np.copy
    for nrow, row in enumerate(matrix):
        # nrow равен номеру строки
        # row содержит саму строку матрицы
        divider = row[nrow] # диагональный элемент
        # делим на диагональный элемент.
        row /= divider
        # теперь надо вычесть приведённую строку из всех нижележащих строчек
        for lower_row in matrix[nrow+1:]:
            factor = lower_row[nrow] # элемент строки в колонке nrow
            lower_row -= factor*row # вычитаем, чтобы получить ноль в колонке nrow
    # все строки матрицы изменились, в принципе, можно и не возвращать
    return matrix

In [4]:
makeTriangleNaive(matrix.copy())

array([[ 1.        ,  1.76315789, -0.31578947,  1.36842105],
       [-0.        ,  1.        ,  0.06800211,  0.49657354],
       [ 0.        ,  0.        ,  1.        ,  0.09309401]])

Для нахождения решения нужно привести матрицу коэффициентов к диагональному виду. Тогда в последнем столбце будет находиться решение.

In [5]:
def makeIdentity(matrix):
    # перебор строк в обратном порядке 
    for nrow in range(len(matrix)-1,0,-1):
        row = matrix[nrow]
        for upper_row in matrix[:nrow]:
            factor = upper_row[nrow]
            # вычитать строки не нужно, так как в row только два элемента отличны от 0:
            # в последней колонке и на диагонали
            
            # вычитание в последней колонке
            upper_row[-1] -= factor*row[-1]
            # вместо вычитания 1*factor просто обнулим коэффициент в соотвествующей колонке. 
            upper_row[nrow] = 0
    return matrix

In [6]:
m1 = makeTriangleNaive(np.copy(matrix))
m2 = makeIdentity(m1)
m2

array([[ 1.        ,  0.        ,  0.        ,  0.53344344],
       [-0.        ,  1.        ,  0.        ,  0.49024295],
       [ 0.        ,  0.        ,  1.        ,  0.09309401]])

После приведения к диагональному виду корни находятся в последнем столбце.

In [7]:
roots = m2[:,-1]
roots

array([0.53344344, 0.49024295, 0.09309401])

**Проверка решения**

Для проверки извлечём матрицу коэффициентов, умножим её справа на столбец корней и вычтем столбец свободных членов исходной матрицы: `Ax - b`. Результат должен оказаться близким к нулю.

In [8]:
coefs = matrix[:,:-1]
coefs

array([[ 3.8,  6.7, -1.2],
       [ 6.4,  1.3, -2.7],
       [ 2.4, -4.5,  3.5]])

In [9]:
# свободные члены в последнем столбце
b = matrix[:,-1]

In [10]:
np.matmul(coefs, roots.T) - b

array([ 0.00000000e+00, -4.44089210e-16, -2.22044605e-16])

**Решение СЛАУ одной функцией**

In [11]:
def gaussSolveNaive(A, b=None):
    """Решает систему линейных алгебраических уравнений Ax=b
    Если b is None, то свободные коэффициенты в последней колонке"""
    shape = A.shape
    assert len(shape) == 2, ("Матрица не двумерная", shape) # двумерная матрица
    A = A.copy()
    if b is not None:
        assert shape[0] == shape[1], ("Матрица не квадратная", shape)
        assert b.shape == (shape[0],), ("Размерность свободных членов не соответствует матрица", shape, b.shape)
        # добавляем свободные члены дополнительным столбцом
        A = np.c_[A, b]
    else:
        # Проверяем, что квадратная плюс столбец
        assert shape[0]+1 == shape[1], ("Неверный формат матрицы", shape)
    makeTriangleNaive(A)
    makeIdentity(A)
    return A[:,-1]

In [12]:
gaussSolveNaive(matrix)

array([0.53344344, 0.49024295, 0.09309401])

In [13]:
gaussSolveNaive(matrix[:,:3], matrix[:,3])

array([0.53344344, 0.49024295, 0.09309401])

Когда на диагонали встречается ноль, происходит деление на ноль. Оно не выбрасывается как исключение, вместо этого возвращается `nan`

In [14]:
gaussSolveNaive(withZero)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


array([nan, nan, nan])

# Решение методом Гаусса с выбором главного элемента

Для того, чтобы избежать проблем с делением на ноль, и вообще повысить устойчивость счета, используется метод Гаусса с выбором главного элемента.

В этом методе перед тем как делить на диагональный элемент среди всех строк, лежащих ниже, находится строка с максимальным по модулю элементом в нужной колонке.

In [15]:
def makeTrianglePivot(matrix):
    for nrow in range(len(matrix)):
        # nrow равен номеру строки
        # np.argmax возвращает номер строки с максимальным элементом в уменьшенной матрице
        # которая начинается со строки nrow. Поэтому нужно прибавить nrow к результату
        pivot = nrow + np.argmax(abs(matrix[nrow:, nrow]))
        if pivot != nrow:
            # swap
            # matrix[nrow], matrix[pivot] = matrix[pivot], matrix[nrow] - не работает.
            # нужно переставлять строки именно так, как написано ниже
            # matrix[[nrow, pivot]] = matrix[[pivot, nrow]]
            matrix[nrow], matrix[pivot] = matrix[pivot], np.copy(matrix[nrow])
        row = matrix[nrow]
        divider = row[nrow] # диагональный элемент
        if abs(divider) < 1e-10:
            # почти нуль на диагонали. Продолжать не имеет смысла, результат счёта неустойчив
            raise ValueError("Матрица несовместна")
        # делим на диагональный элемент.
        row /= divider
        # теперь надо вычесть приведённую строку из всех нижележащих строчек
        for lower_row in matrix[nrow+1:]:
            factor = lower_row[nrow] # элемент строки в колонке nrow
            lower_row -= factor*row # вычитаем, чтобы получить ноль в колонке nrow
    return matrix

In [16]:
makeTrianglePivot(np.array([[1,0,0,1],
                         [0,0,1,2],
                         [0,1,0,3]
                        ], dtype=float))

array([[1., 0., 0., 1.],
       [0., 1., 0., 3.],
       [0., 0., 1., 2.]])

In [17]:
def gaussSolvePivot(A, b=None):
    """Решает систему линейных алгебраических уравнений Ax=b
    Если b is None, то свободные коэффициенты в последней колонке"""
    shape = A.shape
    assert len(shape) == 2, ("Матрица не двумерная", shape) # двумерная матрица
    A = A.copy()
    if b is not None:
        assert shape[0] == shape[1], ("Матрица не квадратная", shape)
        assert b.shape == (shape[0],), ("Размерность свободных членов не соответствует матрица", shape, b.shape)
        # добавляем свободные члены дополнительным столбцом
        A = np.c_[A, b]
    else:
        # Проверяем, что квадратная плюс столбец
        assert shape[0]+1 == shape[1], ("Неверный формат матрицы", shape)
    makeTrianglePivot(A)
    makeIdentity(A)
    return A[:,-1]

In [18]:
gaussSolvePivot(matrix)

array([0.53344344, 0.49024295, 0.09309401])

# Пример матрица 100x100

В примере решается случайная система линейных уравнений с матрицей 100x100

In [19]:
N = 100
randomSle = np.random.rand(N, N)
randomV = np.random.rand(N)

Для начала решим "наивным" способом. Вероятность того, что на диагонали будет нуль, пренебрежимо мала.

In [20]:
randomRoots = gaussSolveNaive(randomSle, randomV)
randomRoots

array([-0.28323687,  0.01498533, -0.76340784,  0.04372762, -0.02944466,
        0.0939925 , -0.66430102, -0.64088742,  0.19871007, -1.25186715,
        0.12364898,  0.00288674, -0.54078095, -0.44144048, -0.12675993,
        1.39815719, -0.73802786, -0.77315503,  1.76471618,  0.07566329,
        0.02421402, -0.06388076, -0.74967479, -0.87184569, -0.69237051,
        0.50374326, -0.15836105,  0.11738275, -0.93076343, -0.72014169,
        1.08157051, -0.06544787, -0.35984007,  0.45626458,  0.86865271,
        1.40484255,  0.53877534,  0.65965865, -0.18887306, -0.26348963,
       -0.5310845 ,  0.30097714, -0.03863381,  1.01484603,  1.24509391,
       -0.90354864,  1.17474189,  0.12957285,  0.44268479, -0.88046586,
        0.86870432, -0.29769049,  0.37484616,  0.7374659 , -1.10519817,
        1.78348463,  0.35455008,  0.4589341 , -1.57447028,  0.82988722,
       -1.29451811, -0.39453367,  1.23635139,  0.40420487,  0.90517399,
        2.008857  , -0.35691669,  0.39778278,  1.14602613, -0.26

In [21]:
randomRoots2 = gaussSolvePivot(randomSle, randomV)

Проверим решение: вычислим максимум модуля в разности `Ax-b`

In [22]:
diff = np.matmul(randomSle, randomRoots) - randomV
np.max(np.abs(diff))

4.702904732312163e-13

In [23]:
diff = np.matmul(randomSle, randomRoots2) - randomV
np.max(np.abs(diff))

7.882583474838611e-15

В обоих случаях `Ax` практически равно `b` - корни найдены успешно. Но решение, найденное методом с выбором главного элемента, построило чуть более точное решение

Сравним найденное решение с решателем, который поставляется с `numpy`:

In [24]:
np_roots = np.linalg.solve(randomSle, randomV)
np.max(np.abs(np.matmul(randomSle, np_roots) - randomV))

6.716849298982197e-15

In [25]:
max(abs(randomRoots - np_roots)), max(abs(randomRoots2 - np_roots))

(3.566088396800282e-13, 3.5416114485542494e-14)

Решения очень близкие. Встроенный решатель построил ещё более точное решение.

## Сравнение времени счёта

In [26]:
%timeit -n10 gaussSolveNaive(randomSle, randomV)

17.9 ms ± 4.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%timeit -n10 gaussSolvePivot(randomSle, randomV)

17.5 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%timeit -n100 np.linalg.solve(randomSle, randomV)

476 µs ± 40.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Методы решения, написанные на чистом пайтоне, считают практически с одинаковой скоростью, и примерно в 40 раз медленее встроенного решателя. Ничего удивительного, встроенный решатель написан на Си.

Ниже представлен трюк, как можно приблизить скорость работы пайтоновского кода к Си-шному, если самые трудоёмкие части кода откомпилировать в машинный код компилятором `numba`

## Ускорение счёта

Для начала обобщим метод решения, выделив функции приведения к треугольному виду и к диагональному виду в параметры.

In [29]:
def generalGauss(A,b, triangleFn=makeTrianglePivot, identityFn=makeIdentity):
    """Решает систему линейных алгебраических уравнений Ax=b
    Если b is None, то свободные коэффициенты в последней колонке"""
    shape = A.shape
    assert len(shape) == 2, ("Матрица не двумерная", shape) # двумерная матрица
    A = A.copy()
    if b is not None:
        assert shape[0] == shape[1], ("Матрица не квадратная", shape)
        assert b.shape == (shape[0],), ("Размерность свободных членов не соответствует матрица", shape, b.shape)
        # добавляем свободные члены дополнительным столбцом
        A = np.c_[A, b]
    else:
        # Проверяем, что квадратная плюс столбец
        assert shape[0]+1 == shape[1], ("Неверный формат матрицы", shape)
    triangleFn(A)
    identityFn(A)
    return A[:,-1]

Проверим, что решение не изменилось

In [30]:
max(abs(generalGauss(randomSle, randomV, triangleFn=makeTriangleNaive, identityFn=makeIdentity) - randomRoots))


0.0

In [31]:
import numba

Немного видоизменённый вариант функции `makeTrianglePivot`, адаптированный к возможностям компилятора `numba`.

Декоратор `numba.njit` предписывает транслировать функцию в чистый машинный код, который не обращается к интерпретатору пайтона. В общем случае это невозможно, но в данном случае у нас все вычисления идут только с `numpy`, а для этого пакета `numba` умеет вызывать Си-инетерфейсы для соответствующих операций - индексирования, присваивания, арифметики.

In [32]:
@numba.njit
def fastMakeTrianglePivot(matrix):
    for nrow in range(len(matrix)):
        pivot = nrow + np.argmax(np.abs(matrix[nrow:, nrow]))
        if pivot != nrow:
            matrix[nrow], matrix[pivot] = matrix[pivot], np.copy(matrix[nrow])
        row = matrix[nrow]
        divider = row[nrow] # диагональный элемент
        if abs(divider) < 1e-10:
            raise ValueError("Матрица несовместна")
        row /= divider
        for lr in range(nrow+1, len(matrix)):
            lower_row = matrix[lr]
            factor = lower_row[nrow]
            lower_row -= factor*row
    return matrix

In [33]:
@numba.njit
def fastMakeIdentity(matrix):
    for nrow in range(len(matrix)-1,0,-1):
        root = matrix[nrow, -1]
        for ur in range(nrow):
            factor = matrix[ur, nrow]
            matrix[ur, -1] -= factor*root
            # вместо вычитания 1*factor просто обнулим коэффициент в соотвествующей колонке. 
            matrix[ur, nrow] = 0.0
    return matrix

Сначала проверим, насколько выросла скорость от замены функции приведения к треугольному виду на скомпилированную
Функцию вызываем два раза. В первом вызове jit-компилятор `numba` транслирует функцию `fastMakeTrianglePivot` в машинный код. Это долгая операция, поэтому результаты измерения времени будут недостоверными.

In [34]:
generalGauss(randomSle, randomV, fastMakeTrianglePivot)

%timeit -n10 -r1 generalGauss(randomSle, randomV, fastMakeTrianglePivot)

5.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


Благодаря компилятору время работы снизилось в 3 раза. Теперь заменим функцию приведения к диагональному виду на скомпилированную.

In [35]:
generalGauss(randomSle, randomV, fastMakeTrianglePivot, fastMakeIdentity)
%timeit -n10 generalGauss(randomSle, randomV, fastMakeTrianglePivot, fastMakeIdentity)

734 µs ± 3.15 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Итого скорость выросла в 20 раз.

Проигрыш по сравнению с решателем на чистом Си, меньше чем в 2 раза